<a href="https://colab.research.google.com/github/Jinal1995/ML/blob/main/Ass_2_ADAAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
# the following package is to import the brest cancer datset. 
from sklearn import datasets
# the following package is to split data in to train and test.
from sklearn.model_selection import train_test_split
from collections import Counter

this cell is for the entropy. we actually implement this for the decision tree. 

In [2]:

def entropy(y):
    hist = np.bincount(y)  # calculate the number of occurances of clasees.
    ps = hist / len(y)   # divided by samples
    E = -np.sum([ps * np.log2(p) for p in ps if p > 0]) # actual formula.
    return E
    

this cell is helper class for the node.

In [3]:
class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature  
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

      # helper class for the node

In [4]:
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):  # random factor
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None  # to traverse the tree.

    def fit(self, X, y):
      # grow tree
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):  # grow tree method
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))   # length of the numpy unique label

        # stopping criteria
        if (depth >= self.max_depth
                or n_labels == 1
                or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        
        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)
    
    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        # weighted average child E
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs 

    def _traverse_tree(self, x, node):
        if node.is_leaf_node(): 
            return node.value

        if x[node.feature] <= node.threshold:    
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)  #counter object
        most_common = counter.most_common(1)[0][0]
        return most_common
        # this will detrmine the most common label

In [5]:
# thia global function is for the accuracy.
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy



the below cell is only for decision tree. as the part of randam forest the tree must be there, so this is for a testing of the tree is working or not.

In [6]:
data = datasets.load_breast_cancer()  #this section loadas the data from sklearn() module. 
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)  # labeled the dataset and split the datset.

clf = DecisionTree(max_depth=10)  # max depth size for the decision tree =10.
clf.fit(X_train, y_train) # fit the data into 
    
y_pred = clf.predict(X_test)  # predict the test data
acc = accuracy(y_test, y_pred)

print ("Accuracy:", acc)

Accuracy: 0.9210526315789473


real implemenatation of random forest algoreith..
it will call descision class to form a tree. 

In [9]:
# global function
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)  # it will make choice between o and number sample and randomly use only the subset.
    return X[idxs], y[idxs]

# global function
def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common


class RandomForest:
    
    def __init__(self, n_trees=10, min_samples_split=2,
                 max_depth=100, n_feats=None):       # this init function gives the trees depth, minimum samples, maximum depth and some optional features for more randomness.
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []  # empty array to save trees.

    def fit(self, X, y): # fit method
        self.trees = [] # empty array to save trees.
        for _ in range(self.n_trees):
            tree = DecisionTree(min_samples_split=self.min_samples_split,
                max_depth=self.max_depth, n_feats=self.n_feats)
            X_samp, y_samp = bootstrap_sample(X, y)  
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)
# training phase finish

    def predict(self, X):  # predict method
        tree_preds = np.array([tree.predict(X) for tree in self.trees]) 
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds] # most common label for each prediction.
        return np.array(y_pred) # convert to the numpy array and return it.

final output with the tree depth of 3

In [10]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

# loding the dataset form sklearn() module.
data = datasets.load_breast_cancer()
X = data.data
y = data.target

# labeled the dataset and split the datset. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = RandomForest(n_trees=3, max_depth=10) # make an instance - RandomForest, tre depth size=3 and max depth is 10. 

# fit the data
clf.fit(X_train, y_train) 
# prdeict the data
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print ("Accuracy:", acc)

Accuracy: 0.9210526315789473


In [11]:
datasets.load_breast_cancer()

{'DESCR': '.. _breast_cancer_dataset:\n\nBreast cancer wisconsin (diagnostic) dataset\n--------------------------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 569\n\n    :Number of Attributes: 30 numeric, predictive attributes and the class\n\n    :Attribute Information:\n        - radius (mean of distances from center to points on the perimeter)\n        - texture (standard deviation of gray-scale values)\n        - perimeter\n        - area\n        - smoothness (local variation in radius lengths)\n        - compactness (perimeter^2 / area - 1.0)\n        - concavity (severity of concave portions of the contour)\n        - concave points (number of concave portions of the contour)\n        - symmetry \n        - fractal dimension ("coastline approximation" - 1)\n\n        The mean, standard error, and "worst" or largest (mean of the three\n        largest values) of these features were computed for each image,\n        resulting in 30 features.  For

the following accuracy is for the random forest


final output with the tree depth of 5

In [12]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = RandomForest(n_trees=5, max_depth=10)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print ("Accuracy:", acc)

Accuracy: 0.9298245614035088


final output with the tree depth of 8

In [13]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = RandomForest(n_trees=8, max_depth=10)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print ("Accuracy:", acc)

Accuracy: 0.9298245614035088
